In [ ]:
#--------------#
###构建模型###
#--------------#

In [1]:
# define the parameters
graph_name = 'GDELT'
root_path = '/home/zhangjs/expriment/TKGist/data/'
dataset = graph_name+'/'

In [2]:
# load graph

import pickle
from graph import Graph
graph = Graph(graph_name, idify=False)
pickle.dump(graph, open(root_path + dataset + "graph_new.pickle", "wb"))
# 4m

1734399it [08:15, 3503.73it/s]


|V| = 7146
|E| = 1734399
|L_V| = 127
|L_E| = 238
|Rule| = 22164


In [3]:
# create a Searcher object to search for a model (set of rules) and build the raw model
from searcher import Searcher
searcher = Searcher(graph)
model = searcher.build_model()
pickle.dump(model, open(root_path + dataset + "static_model_new.pickle", "wb"))
model.print_stats()
#1m40s

ranking rules....
constructing raw model...
Null encoding cost: 12165840.136
Starting pass 1.


22164it [00:16, 1325.92it/s] 


Final number of rules after pass 2: 432
Starting pass 2.


22164it [00:11, 1858.75it/s] 


Final number of rules after pass 3: 434
Starting pass 3.


22164it [00:11, 1958.46it/s] 


Final number of rules after pass 4: 434
----- Model stats -----
L(G,M) = 615613.84
% Bits needed: 5.06
# Rules: 434
% Edges Explained: 99.58
-----------------------


In [4]:
# build the temporal model
temporal_model, candidate_p, candidate_t = searcher.build_temporal_model(model)
pickle.dump(temporal_model, open(root_path + dataset + "temporal_model_new.pickle", "wb"))
temporal_model.print_stats(temporal_model)
#10m

generating chain combinations....


100%|██████████| 371010/371010 [02:15<00:00, 2738.79it/s] 


get candidate rule pairs: 796030
generating riangle combinations....


  3%|▎         | 55/2138 [33:15<20:59:33, 36.28s/it]


get candidate rule riangle: 10047127
Null encoding cost: 145739678.0579
Starting pass 1.


7796030it [03:18, 39248.83it/s]


Final number of combined_rules after pass 2: 27847
----- Model stats -----
L(G,M) = 87782464.46
[87782464.46079448, 145739678.05791223]
% Bits needed: 60.23
# Rules: 27847
% Edges Explained: 89.49
-----------------------


In [ ]:
'''
----- Model stats -----
L(G,M) = 47874454.58
[47874454.580734275, 71251217.27922314]
% Bits needed: 67.19
# Rules: 11385
% Edges Explained: 87.49
-----------------------
generating chain combinations....
100%|██████████| 40839/40839 [01:11<00:00, 567.66it/s] 
get candidate rule pairs: 1041156
generating riangle combinations....
100%|██████████| 304/304 [10:29<00:00,  2.07s/it]
get candidate rule riangle: 7807869
Null encoding cost: 17824148.4108
Starting pass 1.
1041156it [00:33, 31032.89it/s]
Final number of combined_rules after pass 2: 4576
Null encoding cost: 17824148.4108
Starting pass 1.
7807869it [03:31, 36886.14it/s]
Final number of combined_rules after pass 2: 2582
----- Model stats -----
L(G,M) = 14832424.81
[14832424.807377124, 17824148.41079742]
% Bits needed: 83.22
# Rules: 7158
% Edges Explained: 80.86
-----------------------
'''

In [1]:
### directly read preserved files

import pickle
graph_name = 'YAGO'
root_path = '/home/zhangjs/expriment/TKGist/data/'
dataset = graph_name+'/'

graph = pickle.load(open(root_path + dataset + "graph_new.pickle", "rb"))
model = pickle.load(open(root_path + dataset + "static_model_new.pickle", "rb"))
temporal_model = pickle.load(open(root_path + dataset + "temporal_model_new.pickle", "rb"))


In [2]:
# initlize the detection function
import pickle
from anomaly_detector import AnomalyDetector
from model_updater import ModelUpdater

detector = AnomalyDetector(temporal_model)

def read_file(input_file):
    raw_data = []
    for fact in input_file.readlines():
        s, r, o, t = fact.strip().split('	')[:4]
        s = int(s)
        r = int(r)
        o = int(o)
        t = int(t)
        raw_data.append((s, r, o, t))
    return raw_data

valid_pos, test_pos = read_file(open(root_path + dataset + 'valid.txt', 'r')), read_file(open(root_path + dataset + 'test.txt', 'r'))
valid_t_2_C_neg, test_t_2_C_neg = pickle.load(open(root_path + dataset + '/conceptual_errors.pickle', 'rb'))
valid_t_2_T_neg, test_t_2_T_neg = pickle.load(open(root_path + dataset + '/time_errors.pickle', 'rb'))
valid_t_2_M_neg, test_t_2_M_neg = pickle.load(open(root_path + dataset + '/missing_errors.pickle', 'rb'))

valid_t_2_pos = {}
test_t_2_pos = {}
for sample in valid_pos:
    s, r, o, t = sample
    if t not in valid_t_2_pos.keys():
        valid_t_2_pos[t] = []
    valid_t_2_pos[t].append((int(s), int(r), int(o), int(t)))


for sample in test_pos:
    s, r, o, t = sample
    if t not in test_t_2_pos.keys():
        test_t_2_pos[t] = []
    test_t_2_pos[t].append((int(s), int(r), int(o), int(t)))

In [4]:
# detect concept+time+missing anomaly
# next_step_rules_list = list(next_step_rules)
# for rule_pair in next_step_rules_list[:max_rule]:
# #for rule_pair in rules.keys():
        #    self.updater.update_temporal_model(self.temporal_model, rule_pair[0], rule_pair[1], rules[rule_pair], sample_size)

import itertools
import numpy as np
from tqdm import tqdm
import multiprocessing as mp
from multiprocessing.dummy import freeze_support, Manager
from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def get_f1(p, r):
        if (p+r) == 0:
            return 0
        else:
            return 2*(p*r)/(p+r)

def get_f05(p, r):
        if (p+r) == 0:
            return 0
        else:
            return 1.25*(p*r)/(0.25*p+r)
        
def get_metric(pred, y, anomaly_type):
    precision, recall, threshold = precision_recall_curve(y, pred)
    # find the best threshold by F1 score

    f1s = [get_f1(precision[i], recall[i]) for i in range(len(precision))]
    f05s = [get_f05(precision[i], recall[i]) for i in range(len(precision))]
    optimal_index = np.argmax(f05s)
    '''
    while precision[optimal_index] == 1.0:
        f05s[optimal_index] = 0
        optimal_index = np.argmax(f05s)
    '''

    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    

    optimal_P_by_fscore, optimal_R_by_fscore, optimal_F1_by_fscore, optimal_F05_by_fscore, optimal_T_by_fscore = precision[optimal_index], recall[optimal_index], f1s[optimal_index], f05s[optimal_index], threshold[optimal_index]
    optimal_ACC_by_fscore = accuracy_score(y, pred > optimal_T_by_fscore)

    print('----- Result stats -----' + anomaly_type)
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by F1-score:')
    print('P: ' + str(optimal_P_by_fscore))
    print('F05: ' + str(optimal_F05_by_fscore))
    print('AUC: ' + str(auc))

     # find the best threshold by 0.95 Precision
    
    filter_precision = np.array(precision)
    filter_precision[filter_precision < 0.80] = 10
    f1s = [get_f1(precision[i], recall[i]) for i in range(len(precision))]
    f05s = [get_f05(precision[i], recall[i]) for i in range(len(precision))]
    optimal_index = np.argmin(filter_precision)
    if precision[optimal_index] == 1.0:
        filter_precision = np.array(precision)
        filter_precision[filter_precision == 1.0] = 0
        optimal_index = np.argmax(filter_precision)
    
    for ind in range(len(precision)):
        if recall[ind] >= 0.30 and precision[ind] >= 0.80:
            optimal_index = ind
            break

    optimal_P_by_acc, optimal_R_by_acc, optimal_F1_by_acc, optimal_T_by_acc = precision[optimal_index], recall[optimal_index], f1s[optimal_index], threshold[min(optimal_index, len(threshold) - 1)]
    optimal_ACC_by_acc = accuracy_score(y, pred > optimal_T_by_acc)

# init parameters
hop = 2 # ICEWS14 2; ICEWS05 2; YAGO 10 GD 2
span = 2000 # ICEWS14 1000; ICEWS05 2000; YAGO 50 GD 12000
time_specific = False # ICEWS14 F; ICEWS05 F; YAGO F
span_t = 2000 # ICEWS14 200; ICEWS05 2000; YAGO 50 GD 12000
span_m = 2000 # ICEWS14 150; ICEWS05 2000; YAGO 50 GD 12000
max_rule = 10000 # ICEWS14 10000; ICEWS05 10000; YAGO 10000 GD 10000
step = 2 # ICEWS14 2; ICEWS05 2; YAGO 1 GD 2
aux_score = True # ICEWS14 T; ICEWS05 T; YAGO F GD F
aux_score_m = False # ICEWS14 F; ICEWS05 F; YAGO F GD F
distribution_aware = False # ICEWS14 T; ICEWS05 F; YAGO F GD F
and_or = 'and' # ICEWS14 or; ICEWS05 and; YAGO or GD and
update_sample_size = 10 # ICEWS14 10; ICEWS05 10; YAGO 20 GD 5 
update_span = 3000 # ICEWS14 200; ICEWS05 3000; YAGO 20 GD 12000
pred_list_C = [] # for concept anomaly
label_list_C = []

pred_list_T = [] # for time anomaly
label_list_T = []

pred_list_M = [] # for missing anomaly
label_list_M = []

time_specific_pred_list_C = []
time_specific_label_list_C = []
time_specific_pred_list_T = []
time_specific_label_list_T = []
time_specific_pred_list_M = []
time_specific_label_list_M = []

all_proj_rules = []
def update_helper_valid(m):
    raw_score, project_rules = detector.score_edge_temporal((int(m[0]), int(m[1]), int(m[2]), int(m[3])), hop = hop, max_span = span_t, and_or = and_or, max_step = step, aux_score = aux_score, dsa = distribution_aware, max_rule = max_rule, file_type='valid')
    detector.update(m, project_rules, update_sample_size, update_span)

def update_helper_test(x_y):
    m = x_y[0]
    project_rules = x_y[1]
    detector.update(m, project_rules, update_sample_size, update_span)

def concept_detect_helper(fact_label_t):
    m = fact_label_t[0]
    label = fact_label_t[1]
    t = fact_label_t[2]
    raw_score = detector.score_edge((int(m[0]), int(m[1]), int(m[2]), int(t)), hop = hop, max_span = span, and_or = and_or, time_specific = time_specific)
    pred = sigmoid(raw_score)

    return [pred, label]

def temporal_detect_helper(fact_label_t):
    m = fact_label_t[0]
    label = fact_label_t[1]
    t = fact_label_t[2]
    raw_score, project_rules = detector.score_edge_temporal((int(m[0]), int(m[1]), int(m[2]), int(t)), hop = hop, max_span = span_t, and_or = and_or, max_step = step, aux_score = aux_score, dsa = distribution_aware, max_rule = max_rule, file_type='test')
    pred = sigmoid(raw_score)

    return [pred, label, m, project_rules]

def missing_detect_helper(fact_label_t):
    m = fact_label_t[0]
    label = fact_label_t[1]
    t = fact_label_t[2]
    raw_score = detector.score_edge_missing((int(m[0]), int(m[1]), int(m[2]), int(t)), hop_c = hop, max_span_c = span, max_span_t = span_m, and_or = and_or, max_step = step, aux_score = aux_score_m, dsa = distribution_aware)
    pred = sigmoid(-raw_score)

    return [pred, label]

p = mp.Pool(50)
# update model in validate set
for t in tqdm(valid_t_2_pos.keys()):
    pos_samples = valid_t_2_pos[t]
    #p.map(update_helper_valid, pos_samples)

# detect anomalies in test set
for t in tqdm(list(test_t_2_C_neg.keys())[:10]):
    pos_samples = test_t_2_pos[t]
    pos_missing = test_t_2_M_neg[1][t]

    tmp_pred_C = []
    tmp_label_C = []
    tmp_pred_T = []
    tmp_label_T = []
    tmp_pred_M = []
    tmp_label_M = []

    concept_temporal_p = [[pos_samples[i], 0, t] for i in range(len(pos_samples))]
    missing_p = [[pos_missing[i], 1, t] for i in range(len(pos_missing))]
    concept_score_label_p = p.map(concept_detect_helper, concept_temporal_p)
    temporal_score_label_proj_p = p.map(temporal_detect_helper, concept_temporal_p)
    missing_score_label_p = p.map(missing_detect_helper, missing_p)
    for i in range(len(concept_score_label_p)):
        pred_list_C.append(concept_score_label_p[i][0])
        label_list_C.append(concept_score_label_p[i][1])
        tmp_pred_C.append(concept_score_label_p[i][0])
        tmp_label_C.append(concept_score_label_p[i][1])
    
    for i in range(len(temporal_score_label_proj_p)):
        pred_list_T.append(temporal_score_label_proj_p[i][0])
        label_list_T.append(temporal_score_label_proj_p[i][1])
        all_proj_rules.append([temporal_score_label_proj_p[i][2], temporal_score_label_proj_p[i][3]])
        tmp_pred_T.append(temporal_score_label_proj_p[i][0])
        tmp_label_T.append(temporal_score_label_proj_p[i][1])
    
    for i in range(len(missing_score_label_p)):
        pred_list_M.append(missing_score_label_p[i][0])
        label_list_M.append(missing_score_label_p[i][1])
        tmp_pred_M.append(missing_score_label_p[i][0])
        tmp_label_M.append(missing_score_label_p[i][1])
    

    concept_n = [[test_t_2_C_neg[t][i], 1, t] for i in range(len(test_t_2_C_neg[t]))]
    temporal_n = [[test_t_2_T_neg[t][i], 1, t] for i in range(len(test_t_2_T_neg[t]))]
    missing_n = [[test_t_2_M_neg[0][t][i], 0, t] for i in range(len(test_t_2_M_neg[0][t]))]
    concept_score_label_n = p.map(concept_detect_helper, concept_n)
    temporal_score_label_proj_n = p.map(temporal_detect_helper, temporal_n)
    missing_score_label_n = p.map(missing_detect_helper, missing_n)
    

    for i in range(len(concept_score_label_n)):
        pred_list_C.append(concept_score_label_n[i][0])
        label_list_C.append(concept_score_label_n[i][1])
        tmp_pred_C.append(concept_score_label_n[i][0])
        tmp_label_C.append(concept_score_label_n[i][1])
    
    for i in range(len(temporal_score_label_proj_n)):
        pred_list_T.append(temporal_score_label_proj_n[i][0])
        label_list_T.append(temporal_score_label_proj_n[i][1])
        tmp_pred_T.append(temporal_score_label_proj_n[i][0])
        tmp_label_T.append(temporal_score_label_proj_n[i][1])
    
    for i in range(len(missing_score_label_n)):
        pred_list_M.append(missing_score_label_n[i][0])
        label_list_M.append(missing_score_label_n[i][1])
        tmp_pred_M.append(missing_score_label_n[i][0])
        tmp_label_M.append(missing_score_label_n[i][1])
    
    time_specific_pred_list_C.append(tmp_pred_C)
    time_specific_label_list_C.append(tmp_label_C)
    time_specific_pred_list_T.append(tmp_pred_T)
    time_specific_label_list_T.append(tmp_label_T)
    time_specific_pred_list_M.append(tmp_pred_M)
    time_specific_label_list_M.append(tmp_label_M)
    
    # update model in test set
    p.map(update_helper_test, all_proj_rules)
    all_proj_rules = []

get_metric(np.array(pred_list_C), np.array(label_list_C), 'concept') # concept
get_metric(np.array(pred_list_T), np.array(label_list_T), 'time') # time
get_metric(np.array(pred_list_M), np.array(label_list_M), 'missing') # missing
detector.re_fresh()
p.close()
# 19min

100%|██████████| 10/10 [25:38<00:00, 153.86s/it]

----- Result stats -----concept
Number of test samples: 9161
Number of anomaly samples: 2110
-----------------------
Best result by F1-score:
P: 0.4179757407039173
F05: 0.4728696121659318
AUC: 0.7913078444723313
----- Result stats -----time
Number of test samples: 9161
Number of anomaly samples: 2110
-----------------------
Best result by F1-score:
P: 0.23057589334498962
F05: 0.2725112362452859
AUC: 0.47730058793045393
----- Result stats -----missing
Number of test samples: 6330
Number of anomaly samples: 2110
-----------------------
Best result by F1-score:
P: 0.3685467024768726
F05: 0.3980275879850458
AUC: 0.5371391590485389


In [ ]:
print(time_specific_pred_list_C)
print(time_specific_label_list_C)

In [ ]:
time_specific_result_C = []
time_specific_result_M = []
time_specific_result_T = []

def get_auc(pred, y):
    fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    return auc

for i in range(len(time_specific_pred_list_C)):
    time_specific_result_C.append(get_auc(time_specific_pred_list_C[i], time_specific_label_list_C[i]))
    time_specific_result_T.append(get_auc(time_specific_pred_list_T[i], time_specific_label_list_T[i]))
    time_specific_result_M.append(get_auc(time_specific_pred_list_M[i], time_specific_label_list_M[i]))

#print(time_specific_result_M)
pickle.dump([time_specific_result_C, time_specific_result_T, time_specific_result_M], open(root_path + dataset + "auc_time.pickle", "wb"))


In [ ]:
'''
# init parameters
hop = 10 # ICEWS14 2; ICEWS05 2; YAGO 10 GD 2
span = 50 # ICEWS14 1000; ICEWS05 2000; YAGO 50 GD 12000
time_specific = False # ICEWS14 F; ICEWS05 F; YAGO F
span_t = 50 # ICEWS14 200; ICEWS05 2000; YAGO 50 GD 12000
span_m = 50 # ICEWS14 150; ICEWS05 2000; YAGO 50 GD 12000
max_rule = 10000 # ICEWS14 10000; ICEWS05 10000; YAGO 10000 GD 10000
step = 1 # ICEWS14 2; ICEWS05 2; YAGO 1 GD 2
aux_score = False # ICEWS14 T; ICEWS05 T; YAGO F GD F
aux_score_m = False # ICEWS14 F; ICEWS05 F; YAGO F GD F
distribution_aware = False # ICEWS14 T; ICEWS05 F; YAGO F GD F
and_or = 'and' # ICEWS14 or; ICEWS05 and; YAGO or GD and
update_sample_size = 20 # ICEWS14 10; ICEWS05 10; YAGO 20 GD 5 
update_span = 1 # ICEWS14 200; ICEWS05 3000; YAGO 20 GD 12000
pred_list_C = [] # for concept anomaly
label_list_C = []

100%|██████████| 183/183 [02:14<00:00,  1.36it/s]
100%|██████████| 5/5 [00:34<00:00,  6.89s/it]
----- Result stats -----concept
Number of test samples: 26029
Number of anomaly samples: 6004
100%|██████████| 183/183 [02:14<00:00,  1.36it/s]
100%|██████████| 5/5 [00:33<00:00,  6.61s/it]
----- Result stats -----concept
Number of test samples: 26029
Number of anomaly samples: 6004
-----------------------
Best result by F1-score:
P: 0.4029403567447046
F05: 0.4559862486595597
AUC: 0.7685433597742994
----- Result stats -----time
Number of test samples: 26029
Number of anomaly samples: 6004
-----------------------
Best result by F1-score:
P: 0.24585397813357357
F05: 0.28952241339402823
AUC: 0.5401498127340825
----- Result stats -----missing
Number of test samples: 16117
Number of anomaly samples: 6004
-----------------------
Best result by F1-score:
P: 0.437957157784744
F05: 0.45770994867314624
AUC: 0.5093390885525211
'''


'''
hop = 2 # ICEWS14 2; ICEWS05 2; YAGO 10
span = 12000 # ICEWS14 1000; ICEWS05 2000; YAGO 50
span_t = 12000 # ICEWS14 200; ICEWS05 2000; YAGO 50
span_m = 12000 # ICEWS14 150; ICEWS05 2000; YAGO 50
step = 2 # ICEWS14 2; ICEWS05 2; YAGO 1
aux_score = False # ICEWS14 T; ICEWS05 T; YAGO F
aux_score_m = False # ICEWS14 F; ICEWS05 F; YAGO F
distribution_aware = False # ICEWS14 T; ICEWS05 F; YAGO F
and_or = 'and' # ICEWS14 or; ICEWS05 and; YAGO or
update_sample_size = 20 # ICEWS14 10; ICEWS05 10; YAGO 20
update_span = 200 # ICEWS14 200; ICEWS05 3000; YAGO 20

no update max_label = 10, 5, 20 都差不多
only out 性能下降
num_cover_rules += 1 性能下降
num_cover_rules += len(self.temporal_model.model.graph.candidates[rule]['ca_to_size']) +1 性能不变
only 100 rules 微升
Best result by F1-score:

P: 0.8792565539471544
F05: 0.82147139596952
AUC: 0.9001228704018721


P: 0.9336388634280477
F05: 0.7963397938832097
AUC: 0.8798370611945016

P: 0.9236583668419425
F05: 0.8007743103274897
AUC: 0.8835251899489124
'''

In [ ]:
pred_list_T

In [ ]:
'''
and + 3000span
Best result by F1-score:
P: 0.8535224586288416
R: 0.6596988744335623
F: 0.7441975512223276
F0.5: 0.806151977421314
ACC: 0.8962918400427801
Threshold: 0.7424656560607265

# span 3000
Best result by F1-score:
P: 0.8956037840845854
R: 0.5881450080397602
F: 0.7100189703092602
F0.5: 0.8108298738462777
ACC: 0.8901422101903377
Threshold: 0.7424656560607265
----- Result stats -----
Number of test samples: 59841
Number of anomaly samples: 13682
-----------------------
Best result by 0.95 Precision:
P: 0.8960589483085855
R: 0.5866101447156848
F: 0.7090419188126684
ACC: 0.8898915459300479
Threshold: 0.8175744761936437

# span 100
Best result by F1-score:
P: 0.7405571697145787
R: 0.6333869317351264
F: 0.6827923101166089
F0.5: 0.716316746569681
ACC: 0.8653431593723367
Threshold: 0.5621765008857981
----- Result stats -----
Number of test samples: 59841
Number of anomaly samples: 13682
-----------------------
Best result by 0.95 Precision:
P: 0.7414878027756228
R: 0.6287092530331823
F: 0.6804572242218091
ACC: 0.8649420965558731
Threshold: 0.7439624913247581
'''

In [ ]:
'''                 
                    concept    (OK)                                    
no update           Best result by F1-score:                        
                    P: 0.7842592592592592
                    R: 0.7707006369426752
                    F: 0.7774208352455254
                    F0.5: 0.78150950359845
                    ACC: 0.898526491796426
                    Threshold: 0.7685247834990175

                    Best result by 0.95 Precision:
                    P: 0.7842592592592592
                    R: 0.7707006369426752
                    F: 0.7774208352455254
                    ACC: 0.898526491796426
                    Threshold: 0.7685247834990175
'''

'''                 
                    time     (OK)                        
no update           Best result by F1-score:
                    P: 0.5449199607971251
                    R: 0.7588717015468608
                    F: 0.6343411294922989
                    F0.5: 0.5774823431657665
                    ACC: 0.7989340578952868
                    Threshold: 0.923173382831992
                    
                    Best result by 0.95 Precision:
                    P: 0.875
                    R: 0.0031847133757961785
                    F: 0.006346328195829557
                    ACC: 0.7710314557425019
                    Threshold: 0.9512665461461832
'''

'''                 
                    missing     (OK)
no update           Best result by F1-score:
                    P: 0.8570014844136566
                    R: 0.7879890809827116
                    F: 0.8210476416212373
                    F0.5: 0.8422485897685277
                    ACC: 0.7707006369426752
                    Threshold: 0.9175179090752534
                    
                    Best result by 0.95 Precision:
                    P: 0.9321486268174475
                    R: 0.3937670609645132
                    F: 0.5536542459619384
                    ACC: 0.5764331210191083
                    Threshold: 0.9513380924144209
'''

In [ ]:
# detect concept anomaly
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def get_f1(p, r):
        if (p+r) == 0:
            return 0
        else:
            return 2*(p*r)/(p+r)

def get_metric(pred, y):
    precision, recall, threshold = precision_recall_curve(y, pred)
    # find the best threshold by F1 score

    f1s = [get_f1(precision[i], recall[i]) for i in range(len(precision))]
    optimal_index = np.argmax(f1s)

    optimal_P_by_fscore, optimal_R_by_fscore, optimal_F1_by_fscore, optimal_T_by_fscore = precision[optimal_index], recall[optimal_index], f1s[optimal_index], threshold[optimal_index]
    optimal_ACC_by_fscore = accuracy_score(y, pred > optimal_T_by_fscore)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by F1-score:')
    print('P: ' + str(optimal_P_by_fscore))
    print('R: ' + str(optimal_R_by_fscore))
    print('F: ' + str(optimal_F1_by_fscore))
    print('ACC: ' + str(optimal_ACC_by_fscore))
    print('Threshold: ' + str(optimal_T_by_fscore))

    # find the best threshold by 0.95 Precision
    filter_precision = np.array(precision)
    filter_precision[filter_precision < 0.95] = 10
    optimal_index = np.argmin(filter_precision)
    if precision[optimal_index] == 1.0:
        filter_precision = np.array(precision)
        filter_precision[filter_precision == 1.0] = 0
        optimal_index = np.argmax(filter_precision)

    optimal_P_by_acc, optimal_R_by_acc, optimal_F1_by_acc, optimal_T_by_acc = precision[optimal_index], recall[optimal_index], f1s[optimal_index], threshold[min(optimal_index, len(threshold) - 1)]
    optimal_ACC_by_acc = accuracy_score(y, pred > optimal_T_by_acc)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by 0.95 Precision:')
    print('P: ' + str(optimal_P_by_acc))
    print('R: ' + str(optimal_R_by_acc))
    print('F: ' + str(optimal_F1_by_acc))
    print('ACC: ' + str(optimal_ACC_by_acc))
    print('Threshold: ' + str(optimal_T_by_acc))


# test in valid set
for t in tqdm(valid_t_2_pos.keys()):
    pos_samples = valid_t_2_pos[t]
    for m in pos_samples:
        detector.update(m)

# test in test set
hop = 2
span = 1000
pred_list = []
label_list = []
for t in tqdm(test_t_2_pos.keys()):
    pos_samples = test_t_2_pos[t]
    for m in pos_samples:
        raw_score = detector.score_edge(m, hop, span)
        pred = sigmoid(raw_score)
        label = 0
        pred_list.append(pred)
        label_list.append(label)

    neg_samples = test_t_2_C_neg[t] if t in test_t_2_C_neg.keys() else []
    for m in neg_samples:
        raw_score = detector.score_edge((int(m[0]), int(m[1]), int(m[2]), int(t)), hop, span)
        pred = sigmoid(raw_score)
        label = 1
        pred_list.append(pred)
        label_list.append(label)
    for m in pos_samples:
        detector.update(m)
get_metric(np.array(pred_list), np.array(label_list))
detector.re_fresh()



In [ ]:
'''
Best result by F1-score:
P: 0.7874596588289534
R: 0.7770700636942676
F: 0.7822303640943439
ACC: 0.900616574354687
Threshold: 0.5199893401555818
----- Result stats -----
Number of test samples: 9569
Number of anomaly samples: 2198
-----------------------
Best result by 0.95 Precision:
P: 0.7901869158878505
R: 0.7693357597816196
F: 0.7796219455970493
ACC: 0.8999895495872087
Threshold: 0.7685247834990175



# no update
Best result by F1-score:
P: 0.7825886688162137
R: 0.772975432211101
F: 0.7777523460746166
ACC: 0.898526491796426
Threshold: 0.7431680086124811
----- Result stats -----
Number of test samples: 9569
Number of anomaly samples: 2198
-----------------------
Best result by 0.95 Precision:
P: 0.7846225104214914
R: 0.7707006369426752
F: 0.7775992655496902
ACC: 0.898630995924339
Threshold: 0.7685247834990175
'''

In [ ]:
# detect time anomaly
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def get_f1(p, r):
        if (p+r) == 0:
            return 0
        else:
             return 2*(p*r)/(p+r)
            #return 1.25*(p*r)/(0.25*p+r)

def get_f05(p, r):
        if (p+r) == 0:
            return 0
        else:
            return 1.25*(p*r)/(0.25*p+r)

def get_metric(pred, y):
    precision, recall, threshold = precision_recall_curve(y, pred)
    # find the best threshold by F1 score

    f1s = [get_f1(precision[i], recall[i]) for i in range(len(precision))]
    f05s = [get_f05(precision[i], recall[i]) for i in range(len(precision))]
    optimal_index = np.argmax(f05s)

    optimal_P_by_fscore, optimal_R_by_fscore, optimal_F1_by_fscore, optimal_F05_by_fscore, optimal_T_by_fscore = precision[optimal_index], recall[optimal_index], f1s[optimal_index], f05s[optimal_index], threshold[optimal_index]
    optimal_ACC_by_fscore = accuracy_score(y, pred > optimal_T_by_fscore)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by F1-score:')
    print('P: ' + str(optimal_P_by_fscore))
    print('R: ' + str(optimal_R_by_fscore))
    print('F: ' + str(optimal_F1_by_fscore))
    print('F0.5: ' + str(optimal_F05_by_fscore))
    print('ACC: ' + str(optimal_ACC_by_fscore))
    print('Threshold: ' + str(optimal_T_by_fscore))

    # find the best threshold by 0.95 Precision
    filter_precision = np.array(precision)
    filter_precision[filter_precision < 0.95] = 10
    optimal_index = np.argmin(filter_precision)
    if precision[optimal_index] == 1.0:
        filter_precision = np.array(precision)
        filter_precision[filter_precision == 1.0] = 0
        optimal_index = np.argmax(filter_precision)

    optimal_P_by_acc, optimal_R_by_acc, optimal_F1_by_acc, optimal_F05_by_fscore, optimal_T_by_acc = precision[optimal_index], recall[optimal_index], f1s[optimal_index], f05s[optimal_index], threshold[min(optimal_index, len(threshold) - 1)]
    optimal_ACC_by_acc = accuracy_score(y, pred > optimal_T_by_acc)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by 0.95 Precision:')
    print('P: ' + str(optimal_P_by_acc))
    print('R: ' + str(optimal_R_by_acc))
    print('F: ' + str(optimal_F1_by_acc))
    print('F0.5: ' + str(optimal_F05_by_fscore))
    print('ACC: ' + str(optimal_ACC_by_acc))
    print('Threshold: ' + str(optimal_T_by_acc))


# test in valid set
pred_list = []
label_list = []

true_scores = []
false_scores = []
for t in tqdm(valid_t_2_pos.keys()):
    pos_samples = valid_t_2_pos[t]
    for m in pos_samples:
        detector.update(m)


# test in test set

pred_list = []
label_list = []

raw_score_list_p = []
raw_score_list_n = []
for t in tqdm(test_t_2_pos.keys()):
    pos_samples = test_t_2_pos[t]
    for m in pos_samples:
        raw_score = detector.score_edge_temporal(m)
        raw_score_list_p.append(raw_score)
        pred = sigmoid(raw_score)
        label = 0
        pred_list.append(pred)
        label_list.append(label)

    neg_samples = test_t_2_T_neg[t] if t in test_t_2_T_neg.keys() else []
    for m in neg_samples:
        raw_score = detector.score_edge_temporal((int(m[0]), int(m[1]), int(m[2]), int(t)))
        raw_score_list_n.append(raw_score)
        pred = sigmoid(raw_score)
        label = 1
        pred_list.append(pred)
        label_list.append(label)
    for m in pos_samples:
        detector.update(m)
get_metric(np.array(pred_list), np.array(label_list))
detector.re_fresh()

In [ ]:
Best result by F1-score:
P: 0.6872586872586872
R: 0.7288444040036397
F: 0.7074409361890042
F0.5: 0.6951918069779552
ACC: 0.8612185181314662
Threshold: 0.9241418199787566

# no update
Best result by F1-score:
P: 0.5421412300683371
R: 0.7579617834394905
F: 0.6321381142098274
F0.5: 0.5748792270531401
ACC: 0.7972619918486781
Threshold: 0.9214404114882956


#pair单独排序；时间衰减
Best result by 0.95 Precision:
P: 0.3124006359300477
R: 0.7151956323930846
F: 0.4348547717842323
F0.5: 0.3520558989518947
ACC: 0.5728916292193542
Threshold: 0.5007204605964806

#pair+triangle混合排序；允许内部二阶递归；1/计数;triange生成500_10；有时间间隔限制
Best result by 0.95 Precision:
P: 0.31661600810536983
R: 0.8530482256596906
F: 0.46182266009852224
F0.5: 0.36216487676736464
ACC: 0.5432124568920472
Threshold: 0.734699799197555

#pair+triangle单独排序但混和选择会导致二阶规则几乎不被选入模型
#pair+triangle单独排序且分别选择；只允许内部二阶递归时；性能下降
#pair+triangle单独排序且分别选择；允许内部，外部二阶递归时；性能下降

In [ ]:
# detect missing anomaly
import numpy as np
from tqdm import tqdm
from sklearn.metrics import precision_recall_curve, accuracy_score, f1_score

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def get_f1(p, r):
        if (p+r) == 0:
            return 0
        else:
             return 2*(p*r)/(p+r)

def get_f05(p, r):
        if (p+r) == 0:
            return 0
        else:
            return 1.25*(p*r)/(0.25*p+r)

def get_metric(pred, y):
    precision, recall, threshold = precision_recall_curve(y, pred)
    # find the best threshold by F1 score

    f1s = [get_f1(precision[i], recall[i]) for i in range(len(precision))]
    f05s = [get_f05(precision[i], recall[i]) for i in range(len(precision))]
    optimal_index = np.argmax(f05s)

    optimal_P_by_fscore, optimal_R_by_fscore, optimal_F1_by_fscore, optimal_F05_by_fscore, optimal_T_by_fscore = precision[optimal_index], recall[optimal_index], f1s[optimal_index], f05s[optimal_index], threshold[optimal_index]
    optimal_ACC_by_fscore = accuracy_score(y, pred > optimal_T_by_fscore)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by F1-score:')
    print('P: ' + str(optimal_P_by_fscore))
    print('R: ' + str(optimal_R_by_fscore))
    print('F: ' + str(optimal_F1_by_fscore))
    print('F0.5: ' + str(optimal_F05_by_fscore))
    print('ACC: ' + str(optimal_ACC_by_fscore))
    print('Threshold: ' + str(optimal_T_by_fscore))

    # find the best threshold by 0.95 Precision
    filter_precision = np.array(precision)
    filter_precision[filter_precision < 0.95] = 10
    optimal_index = np.argmin(filter_precision)
    if precision[optimal_index] == 1.0:
        filter_precision = np.array(precision)
        filter_precision[filter_precision == 1.0] = 0
        optimal_index = np.argmax(filter_precision)

    optimal_P_by_acc, optimal_R_by_acc, optimal_F1_by_acc, optimal_F05_by_fscore, optimal_T_by_acc = precision[optimal_index], recall[optimal_index], f1s[optimal_index], f05s[optimal_index], threshold[min(optimal_index, len(threshold) - 1)]
    optimal_ACC_by_acc = accuracy_score(y, pred > optimal_T_by_acc)

    print('----- Result stats -----')
    print('Number of test samples: ' + str(len(y)))
    print('Number of anomaly samples: ' + str(sum(y)))
    print('-----------------------')
    print('Best result by 0.95 Precision:')
    print('P: ' + str(optimal_P_by_acc))
    print('R: ' + str(optimal_R_by_acc))
    print('F: ' + str(optimal_F1_by_acc))
    print('F0.5: ' + str(optimal_F05_by_fscore))
    print('ACC: ' + str(optimal_ACC_by_acc))
    print('Threshold: ' + str(optimal_T_by_acc))


# test in valid set
pred_list = []
label_list = []

true_scores = []
false_scores = []
for t in tqdm(valid_t_2_pos.keys()):
    pos_samples = valid_t_2_pos[t]
    for m in pos_samples:
        detector.update(m)
         

# test in test set

pred_list = []
label_list = []

raw_score_list_p = []
raw_score_list_n = []
missing_num = 0
error_num = 0 
for t in tqdm(test_t_2_pos.keys()):
    pos_samples = test_t_2_pos[t]
    pos_missing = test_t_2_M_neg[1][t]
    for m in pos_missing:
        missing_num+=1
        raw_score = detector.score_edge_missing((int(m[0]), int(m[1]), int(m[2]), int(t)))
        raw_score_list_p.append(raw_score)
        pred = sigmoid(raw_score)
        label = 0
        pred_list.append(pred)
        label_list.append(label)

    neg_samples = test_t_2_M_neg[0][t]# if t in test_t_2_M_neg[0].keys() else []
    for m in neg_samples:
        error_num+=1
        raw_score = detector.score_edge_missing((int(m[0]), int(m[1]), int(m[2]), int(t)))
        raw_score_list_n.append(raw_score)
        pred = sigmoid(raw_score)
        label = 1
        pred_list.append(pred)
        label_list.append(label)
    for m in pos_samples:
        detector.update(m)
get_metric(np.array(pred_list), np.array(label_list))
detector.re_fresh()

In [ ]:
Best result by 0.95 Precision:
P: 0.9362050163576882
R: 0.390582347588717
F: 0.5512038523274478
F0.5: 0.7317592908285032
ACC: 0.3333333333333333
Threshold: 0.9525741268224334

Best result by 0.95 Precision:
P: 0.9331536388140161
R: 0.3937670609645132
F: 0.5538313869780835
F0.5: 0.7324813811780635
ACC: 0.5767364270548984
Threshold: 0.9514022157375411

# no update
Best result by 0.95 Precision:
P: 0.932972972972973
R: 0.39262966333030025
F: 0.5526737111751522
F0.5: 0.731603933536792
ACC: 0.5759781619654231
Threshold: 0.9308615796566533

In [ ]:
print(list(temporal_model.rule_to_time.keys())[:10])

In [ ]:
print(temporal_model.rule_to_time[(((43, 0, 45, 'in'), (45, 15, 44, 'out')), (44, 0, 43, 'out'))])

In [ ]:
print(graph.e_2_t_list[1000].keys())
print(graph.e_2_t_list[1000][296])

In [ ]:
#################测试部分#######################

In [ ]:
from tqdm import tqdm
triple_sample_size = 100
rule_sample_size = 5
max_time = max(list(graph.t_2_triple.keys()))

triangle_list = dict()
for time in tqdm(sorted(list(graph.t_2_triple.keys()))):
    triple_list = graph.t_2_triple[time]
    for triple in list(triple_list)[:100]:
        sub = triple[0]
        pred = triple[1]
        obj = triple[2]
        next_time_list = set([time])
        if time in graph.e_2_t_list[sub].keys():
            next_time_list.add(graph.e_2_t_list[sub][time])
        if time in graph.e_2_t_list[obj].keys():
            next_time_list.add(graph.e_2_t_list[obj][time])

        for in_time in next_time_list:
            triple_list_2rd = set()
            delete_set = set()
            if sub in graph.t_e_2_triple[in_time].keys():
                triple_list_2rd = triple_list_2rd | graph.t_e_2_triple[in_time][sub] 
                delete_set = graph.t_e_2_triple[in_time][sub]
            if obj in graph.t_e_2_triple[in_time].keys():
                triple_list_2rd = triple_list_2rd | graph.t_e_2_triple[in_time][obj]
                delete_set = delete_set & graph.t_e_2_triple[in_time][obj]
            
            triple_list_2rd = triple_list_2rd - delete_set

            for triple_2rd in list(triple_list_2rd)[:100]:
                sub_2 = triple_2rd[0]
                pred_2 = triple_2rd[1]
                obj_2 = triple_2rd[2]
                
                co_e = list(set([sub, obj]) & set([sub_2, obj_2]))[0]
                candidate_e_1 = sub if obj == co_e else obj
                candidate_e_2 = sub_2 if obj_2 == co_e else obj_2
                
                next_time_list = set([in_time])
                if in_time in graph.e_2_t_list[candidate_e_1].keys():
                    next_time_list.add(graph.e_2_t_list[candidate_e_1][in_time])
                if in_time in graph.e_2_t_list[candidate_e_2].keys():
                    next_time_list.add(graph.e_2_t_list[candidate_e_2][in_time])

                for close_time in next_time_list:
                    triple_list_3rd = set()
                    if (candidate_e_1, candidate_e_2) in graph.t_pair_2_triple[close_time].keys():
                        triple_list_3rd = triple_list_3rd | graph.t_pair_2_triple[close_time][(candidate_e_1, candidate_e_2)]
                    if (candidate_e_2, candidate_e_1) in graph.t_pair_2_triple[close_time].keys():
                        triple_list_3rd = triple_list_3rd | graph.t_pair_2_triple[close_time][(candidate_e_2, candidate_e_1)]
                    
                    for triple_3rd in list(triple_list_3rd)[:100]:
                        candidate_1_rule = sorted(graph.triple_to_rule[triple], reverse=True, key=lambda g: len(graph.candidates[g]['triples']))[: rule_sample_size]
                        candidate_2_rule = sorted(graph.triple_to_rule[triple_2rd], reverse=True, key=lambda g: len(graph.candidates[g]['triples']))[: rule_sample_size]
                        candidate_3_rule = graph.triple_to_rule[triple_3rd]

                        
                        for rule_1 in candidate_1_rule:
                            for rule_2 in candidate_2_rule:
                                co_e_list = list(set([rule_1[0], rule_1[2]]) & set([rule_2[0], rule_2[2]]))
                                if len(co_e_list) != 1:
                                    continue
                                co_e = co_e_list[0]
                                rule_3_e_1 = rule_1[0] if rule_1[2] == co_e else rule_1[2]
                                rule_3_e_2 = rule_2[0] if rule_2[2] == co_e else rule_2[2]
                                combined_rules = [(rule_3_e_1, triple_3rd[1], rule_3_e_2, 'out'), (rule_3_e_2, triple_3rd[1], rule_3_e_1, 'out'), (rule_3_e_1, triple_3rd[1], rule_3_e_2, 'in'), (rule_3_e_2, triple_3rd[1], rule_3_e_1, 'in')]
                                for rule_3 in combined_rules:
                                    if rule_3 in candidate_3_rule:
                                        if (rule_1, rule_2, rule_3) not in triangle_list.keys():
                                            triangle_list[(rule_1, rule_2, rule_3)] = set()
                                        triangle_list[(rule_1, rule_2, rule_3)].add((triple_3rd[0], triple_3rd[1], triple_3rd[2]))
                        


In [ ]:
from tqdm import tqdm
triple_sample_size = 1000
rule_sample_size = 5
max_time = max(list(graph.t_2_triple.keys()))
covered_set = set()

triangle_list = dict()
for time in tqdm(sorted(list(graph.t_2_triple.keys()), reverse=True)):
    target_triple_list = list(graph.t_2_triple[time])
    for target_triple in target_triple_list[:triple_sample_size]:
        if target_triple in covered_set:
            continue
        sub = target_triple[0]
        pred = target_triple[1]
        obj = target_triple[2]
        pre_time_list = set([time])
        if time in graph.e_2_t_list[sub].keys():
            pre_time_list.add(graph.e_2_t_list[sub][time])
        if time in graph.e_2_t_list[obj].keys():
            pre_time_list.add(graph.e_2_t_list[obj][time])

        for in_time in pre_time_list:
            triple_list_2rd = set()
            delete_set = set()
            if sub in graph.t_e_2_triple[in_time].keys():
                triple_list_2rd = triple_list_2rd | graph.t_e_2_triple[in_time][sub] 
                delete_set = graph.t_e_2_triple[in_time][sub]
            if obj in graph.t_e_2_triple[in_time].keys():
                triple_list_2rd = triple_list_2rd | graph.t_e_2_triple[in_time][obj]
                delete_set = delete_set & graph.t_e_2_triple[in_time][obj]
            
            triple_list_2rd = triple_list_2rd - delete_set
            
            for triple_2rd in list(triple_list_2rd)[:triple_sample_size]:
                sub_2 = triple_2rd[0]
                pred_2 = triple_2rd[1]
                obj_2 = triple_2rd[2]

                co_e = list(set([sub, obj]) & set([sub_2, obj_2]))[0]
                candidate_e_1 = sub if obj == co_e else obj
                candidate_e_2 = sub_2 if obj_2 == co_e else obj_2

                pre_time_list = set([in_time])
                if in_time in graph.e_2_t_list[candidate_e_1].keys():
                    pre_time_list.add(graph.e_2_t_list[candidate_e_1][in_time])
                if in_time in graph.e_2_t_list[candidate_e_2].keys():
                    pre_time_list.add(graph.e_2_t_list[candidate_e_2][in_time])

                for close_time in pre_time_list:
                    triple_list_3rd = set()
                    if (candidate_e_1, candidate_e_2) in graph.t_pair_2_triple[close_time].keys():
                        triple_list_3rd = triple_list_3rd | graph.t_pair_2_triple[close_time][(candidate_e_1, candidate_e_2)]
                    if (candidate_e_2, candidate_e_1) in graph.t_pair_2_triple[close_time].keys():
                        triple_list_3rd = triple_list_3rd | graph.t_pair_2_triple[close_time][(candidate_e_2, candidate_e_1)]
                    
                    for triple_3rd in list(triple_list_3rd)[:100]:
                        candidate_1_rule = sorted(graph.triple_to_rule[target_triple], reverse=True, key=lambda g: len(graph.candidates[g]['triples']))[: rule_sample_size]
                        candidate_2_rule = sorted(graph.triple_to_rule[triple_2rd], reverse=True, key=lambda g: len(graph.candidates[g]['triples']))[: rule_sample_size]
                        candidate_3_rule = graph.triple_to_rule[triple_3rd]

                        
                        for rule_1 in candidate_1_rule:
                            for rule_2 in candidate_2_rule:
                                co_e_list = list(set([rule_1[0], rule_1[2]]) & set([rule_2[0], rule_2[2]]))
                                if len(co_e_list) != 1:
                                    continue
                                co_e = co_e_list[0]
                                rule_3_e_1 = rule_1[0] if rule_1[2] == co_e else rule_1[2]
                                rule_3_e_2 = rule_2[0] if rule_2[2] == co_e else rule_2[2]
                                combined_rules = [(rule_3_e_1, triple_3rd[1], rule_3_e_2, 'out'), (rule_3_e_2, triple_3rd[1], rule_3_e_1, 'out'), (rule_3_e_1, triple_3rd[1], rule_3_e_2, 'in'), (rule_3_e_2, triple_3rd[1], rule_3_e_1, 'in')]
                                for rule_3 in combined_rules:
                                    if rule_3 in candidate_3_rule:
                                        if (rule_1, rule_2, rule_3) not in triangle_list.keys():
                                            triangle_list[(rule_1, rule_2, rule_3)] = set()
                                        covered_set.add((sub, pred, obj, time))
                                        triangle_list[(rule_1, rule_2, rule_3)].add((sub, pred, obj, time))

In [ ]:
from tqdm import tqdm
triple_sample_size = 1000
rule_sample_size = 5
max_time = max(list(graph.t_2_triple.keys()))
covered_set = set()

def get_rule_triangle(target_triple, sub_triple, obj_triple, triangle_list, covered_set):
    tmp_set = set()
    tar_sub, tar_pred, tar_obj, tar_dir = target_triple[0], target_triple[1], target_triple[2], target_triple[3]
    co_e_list = list(set([sub_triple[0], sub_triple[2]]) & set([obj_triple[0], obj_triple[2]]))
    co_e = co_e_list[0]
    if sub_triple[0] == co_e:
        sub_triple_type = 'co_sub'
    else:
        sub_triple_type = 'sub_co'
    sub_pred, sub_dir = sub_triple[1], sub_triple[3]

    if obj_triple[0] == co_e:
        obj_triple_type = 'co_obj'
    else:
        obj_triple_type = 'obj_co'
    obj_pred, obj_dir = obj_triple[1], obj_triple[3]

    for type_1 in graph.node_to_labels[tar_sub][:rule_sample_size]:
        for type_2 in graph.node_to_labels[tar_obj][:rule_sample_size]:
            target_rules = (type_1, tar_pred, type_2, tar_dir)
            for type_3 in graph.node_to_labels[co_e][:rule_sample_size]:
                
                if sub_triple_type == 'co_sub':
                    sub_rules = (type_3, sub_pred, type_1, sub_dir)
                else:
                    sub_rules = (type_1, sub_pred, type_3, sub_dir)

                if obj_triple_type == 'co_obj':
                    obj_rules = (type_3, obj_pred, type_1, obj_dir)
                else:
                    obj_rules = (type_1, obj_pred, type_3, obj_dir)

                rule_triangle = (sub_rules, obj_rules, target_rules)
                if rule_triangle not in triangle_list.keys():
                    triangle_list[rule_triangle] = set()
                covered_set.add(target_triple)
                triangle_list[rule_triangle].add(target_triple)

triangle_list = dict()
for time in tqdm(sorted(list(graph.t_2_triple.keys()), reverse=True)):
    target_triple_list = list(graph.t_2_triple[time])
    for target_triple in target_triple_list:
        if target_triple in covered_set:
            continue
        sub = target_triple[0]
        pred = target_triple[1]
        obj = target_triple[2]
        pre_time_list_sub = set([time])
        pre_time_list_obj = set([time])
        if time in graph.e_2_t_list[sub].keys():
            pre_time_list_sub = list(pre_time_list_sub | set(graph.e_2_t_list[sub][time]))[:10]
        if time in graph.e_2_t_list[obj].keys():
            pre_time_list_obj = list(pre_time_list_obj | set(graph.e_2_t_list[obj][time]))[:10]

        triple_list_sub = set()
        triple_list_obj = set()
        delete_set = set()
        for in_time in pre_time_list_sub:
            triple_list_sub = triple_list_sub | graph.t_e_2_triple[in_time][sub]
            delete_set = graph.t_e_2_triple[in_time][sub]
        for in_time in pre_time_list_obj:
            triple_list_obj = triple_list_obj | graph.t_e_2_triple[in_time][obj]
            delete_set = delete_set & graph.t_e_2_triple[in_time][obj]
        
        triple_list_sub = list(triple_list_sub - delete_set)
        triple_list_obj = list(triple_list_obj - delete_set)
        
        for sub_triple in triple_list_sub[:triple_sample_size]:
            for obj_triple in triple_list_obj[:triple_sample_size]:
                if len(set([sub_triple[0], sub_triple[2], target_triple[0], target_triple[2]])) != 3:
                    continue
                if len(set([obj_triple[0], obj_triple[2], target_triple[0], target_triple[2]])) != 3:
                    continue
                if len(set([sub_triple[0], sub_triple[2], obj_triple[0], obj_triple[2]])) != 3:
                    continue
                get_rule_triangle(target_triple, sub_triple, obj_triple, triangle_list, covered_set)


In [ ]:
# rule sample -->9337468  100%|██████████| 304/304 [05:52<00:00,  1.95s/it]
# rule+triple sample --> 3007838
print(len(triangle_list))
#print(triangle_list)
print(len(covered_set), graph.fact_num)

In [ ]:
(44, 16, 50, 'in') in graph.candidates.keys()

In [ ]:
(50, 16, 44, 'in') in graph.candidates.keys()

In [ ]:
import pickle
graph_name = 'GDELT'
root_path = '/home/zhangjs/expriment/TKGist/data/'
dataset = graph_name+'/'

def read_file(input_file):
    raw_data = []
    for fact in input_file.readlines():
        s, r, o, t = fact.strip().split('	')[:4]
        s = int(s)
        r = int(r)
        o = int(o)
        t = int(t)
        raw_data.append((s, r, o, t))
    return raw_data

valid_t_2_C_neg, test_t_2_C_neg = pickle.load(open(root_path + dataset + '/conceptual_errors.pickle', 'rb'))
valid_t_2_T_neg, test_t_2_T_neg = pickle.load(open(root_path + dataset + '/time_errors.pickle', 'rb'))
valid_t_2_M_neg, test_t_2_M_neg = pickle.load(open(root_path + dataset + '/missing_errors.pickle', 'rb'))

num_anom_C = 0
for time in test_t_2_C_neg.keys():
    num_anom_C += len(test_t_2_C_neg[time])

num_anom_T = 0
for time in test_t_2_T_neg.keys():
    num_anom_T += len(test_t_2_T_neg[time])

num_anom_M = 0
for time in test_t_2_M_neg[1].keys():
    num_anom_M += len(test_t_2_M_neg[1][time])

print([num_anom_C, num_anom_T, num_anom_M])
